## HC3N Pyspeckit Fitting Results in LaTex + Calculations

Author: @millbets code adapted by @alieber1 for new results

In [2]:
# Import packages
import numpy as np                          # I use this for maths and sometimes arrays.
                                            # "np" is just an abbreviation since we call this package so often!
import pylab                                # I use this for arrays and plotting
import matplotlib                           # Another plotting package
import matplotlib.gridspec as gridspec      # If there is a task you use a lot, importing it like this 
                                            # keeps you from having to constantly type "matplotlib.gridspec"
                                            # every time you call that task!
import scipy
import matplotlib.pyplot as plt
from radio_beam import Beam

import astropy.io.fits as fits              # I use this to import FITS images. 
                                            # Astropy should be included in your conda installation
from astropy.wcs import WCS                 # This one helps us add coordinates to images
from astropy import units as u              # This one helps with unit conversion
from astropy import coordinates as coord
from astropy import log

import regions

import pyspeckit as psk                     # I use this to do spectral line fitting
                                            # You probably don't have it installed; 
                                            # to install, type 'pip install pyspeckit' in a terminal window
        
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes

from reproject import reproject_interp      # Reproject is another useful package you should install
from reproject.mosaicking import find_optimal_celestial_wcs 

import astropy.io.fits as pyfits

                                            # Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

                                            # Super important!! 
                                            # This embeds plots in the Jupyter window 
                                            # (instead of showing them as pop-ups)
%matplotlib inline                             

matplotlib.rc('font', family='sans-serif')  # Set plot fonts
#matplotlib.rc('text', usetex=True)      
matplotlib.rc('text', usetex=False)        # This is a quick fix if you don't have latex on your computer

import os
os.environ["PATH"] += os.pathsep + '/Library/TeX'


In [3]:
sources = pylab.array(['1','2','3','4','5','6','7SW','7','8W','8','9','9NE','10','10NE','11','12','13','14'])

# Source Right Ascensions
RAs =  pylab.array([11.886669,11.886749,11.886829,11.8872162,11.8874364,11.88755,11.887466,\
                    11.887567,11.887861,11.887978,11.887984,11.888092,11.8881464,11.888209,\
                    11.8882,11.888264,11.888322,11.8887287])

# Source Declinations
Decs = pylab.array([-25.289232,-25.289240,-25.289200,-25.2889692,-25.288806,-25.288737,-25.288682,\
                    -25.288628,-25.288251,-25.288244,-25.288389,-25.288297,-25.2880799,-25.288021,\
                    -25.288175,-25.288095,-25.287989,-25.2876482])  


In [4]:
fit1 = np.loadtxt('HC3N_v7_1e_fitting_results_A.txt', delimiter=' ')
linezz = fit1[0]
print(linezz)
print(linezz[0])

[3.09968980e+02 1.85999359e+00 1.13363166e+01 1.85999359e+00
 7.24004011e-01 1.02872423e-01]
309.96898046457443


In [6]:

print( '\\begin{table*}[t]')
print( '\caption{Observed Line Parameters}')
print( '\centering')
print( '\\begin{tabular}{llllll}')
print( '\hline\hline')
print( '       &  RA       & Dec       & \multicolumn{3}{c}{----------- HC$_3$N v$_7$=1e -----------} \\\\')
print( 'Source & ($\degr$) & ($\degr$) & v$_{cen}$    &  $\\sigma_{\mathrm{FWHM}}$ &  Peak T$_B$      \\\\')
print( '       &           &           &(km s$^{-1}$) & (km s$^{-1}$)          & (K)     \\\\'   )              
print( '\hline')

q = 0
for i,source in enumerate(sources):
    line = fit1[i]
    
    Fa = line[4]*line[2]*2.3548   # Peak temp (Amp) * Sigma * 2.3548
    Fluxa = line[4]*line[2]*np.sqrt(2*np.pi) # peak Temp * sigma * sqrt(2pi)
    flux_err_a = np.sqrt((line[5]/line[4])**2+(line[3]/line[2])**2) * Fa  # Error propagation

    Ypm = 5
    Ypm_err = 5

    if (i==0)+(i==1)+(i==2)+(i==3)+(i==9)+(i==16)+(i==17):
            print( source+'&  %1.5f & %1.5f & %2.1f $\pm$ %1.1f & %i $\pm$ %i & %2.1f $\pm$ %1.1f \\\\' %
                    (RAs[i],Decs[i],line[0],line[1],line[2]*2.3548,line[3]*2.3548,line[4],line[5]))
 
    
        
print( '\hline\hline')
print( '\end{tabular}')
print( '\label{params}')
print( '\\end{table*}')
print( ' ')
print( ' '  )

\begin{table*}[t]
\caption{Observed Line Parameters}
\centering
\begin{tabular}{llllll}
\hline\hline
       &  RA       & Dec       & \multicolumn{3}{c}{----------- HC$_3$N v$_7$=1e -----------} \\
Source & ($\degr$) & ($\degr$) & v$_{cen}$    &  $\sigma_{\mathrm{FWHM}}$ &  Peak T$_B$      \\
       &           &           &(km s$^{-1}$) & (km s$^{-1}$)          & (K)     \\
\hline
1&  11.88667 & -25.28923 & 310.0 $\pm$ 1.9 & 26 $\pm$ 4 & 0.7 $\pm$ 0.1 \\
2&  11.88675 & -25.28924 & 310.0 $\pm$ 1.0 & 22 $\pm$ 2 & 1.3 $\pm$ 0.1 \\
3&  11.88683 & -25.28920 & 305.2 $\pm$ 1.1 & 31 $\pm$ 2 & 1.4 $\pm$ 0.1 \\
4&  11.88722 & -25.28897 & 251.7 $\pm$ 1.1 & 20 $\pm$ 2 & 1.1 $\pm$ 0.1 \\
8&  11.88798 & -25.28824 & 303.3 $\pm$ 0.9 & 22 $\pm$ 2 & 1.4 $\pm$ 0.1 \\
13&  11.88832 & -25.28799 & 254.5 $\pm$ 0.8 & 29 $\pm$ 1 & 2.1 $\pm$ 0.1 \\
14&  11.88873 & -25.28765 & 196.6 $\pm$ 0.4 & 48 $\pm$ 0 & 5.0 $\pm$ 0.1 \\
\hline\hline
\end{tabular}
\label{params}
\end{table*}
 
 


In [11]:
Cube3_A = np.loadtxt('HC3N_nonvib_fitting_results_A.txt', delimiter=' ')
Cube3_A_fit = Cube3_A[0]
print(Cube3_A_fit)
print(Cube3_A_fit[0])

Cube3_B = np.loadtxt('HC3N_nonvib_fitting_results_B.txt', delimiter=' ')
Cube3_B_fit = Cube3_B[0]

[2.98553334e+02 1.54250510e-01 1.92073814e+01 1.54250510e-01
 9.77327247e+00 6.79720432e-02]
298.5533344613625


In [13]:
print( '\\begin{table*}[t]')
print( '\caption{Observed Line Parameters}')
print( '\centering')
print( '\\begin{tabular}{llllll}')
print( '\hline\hline')
print( '       &  RA       & Dec       & \multicolumn{3}{c}{----------- HC$_3$N v=0 J=11-10 -----------} \\\\')
print( 'Source & ($\degr$) & ($\degr$) & v$_{cen}$    &  $\\sigma_{\mathrm{FWHM}}$ &  Peak T$_B$      \\\\')
print( '       &           &           &(km s$^{-1}$) & (km s$^{-1}$)          & (K)     \\\\'   )              
print( '\hline')

q = 0
for i,source in enumerate(sources):
    line = Cube3_A[i]
    line2 = Cube3_B[i]

    
    Fa = line[4]*line[2]*2.3548   # Peak temp (Amp) * Sigma * 2.3548
    Fluxa = line[4]*line[2]*np.sqrt(2*np.pi) # peak Temp * sigma * sqrt(2pi)
    flux_err_a = np.sqrt((line[5]/line[4])**2+(line[3]/line[2])**2) * Fa  # Error propagation

    Ypm = 5
    Ypm_err = 5
    
    print( source+'&  %1.5f & %1.5f & %2.1f $\pm$ %1.1f & %i $\pm$ %i & %2.1f $\pm$ %1.1f \\\\' %
          (RAs[i],Decs[i],line[0],line[1],line[2]*2.3548,line[3]*2.3548,line[4],line[5]))
            
    if line2[0]!=0: 
            print( '& & & %2.1f $\pm$ %1.1f & %i $\pm$ %i & %2.1f $\pm$ %1.1f \\\\' %
                (line2[0],line2[1],line2[2]*2.3548,line2[3]*2.3548,line2[4],line2[5]))
 
    
        
print( '\hline\hline')
print( '\end{tabular}')
print( '\label{params}')
print( '\\end{table*}')
print( ' ')

\begin{table*}[t]
\caption{Observed Line Parameters}
\centering
\begin{tabular}{llllll}
\hline\hline
       &  RA       & Dec       & \multicolumn{3}{c}{----------- HC$_3$N v=0 J=11-10 -----------} \\
Source & ($\degr$) & ($\degr$) & v$_{cen}$    &  $\sigma_{\mathrm{FWHM}}$ &  Peak T$_B$      \\
       &           &           &(km s$^{-1}$) & (km s$^{-1}$)          & (K)     \\
\hline
1&  11.88667 & -25.28923 & 298.6 $\pm$ 0.2 & 45 $\pm$ 0 & 9.8 $\pm$ 0.1 \\
2&  11.88675 & -25.28924 & 296.1 $\pm$ 0.1 & 45 $\pm$ 0 & 12.0 $\pm$ 0.1 \\
3&  11.88683 & -25.28920 & 286.9 $\pm$ 0.2 & 50 $\pm$ 0 & 12.1 $\pm$ 0.1 \\
4&  11.88722 & -25.28897 & 259.4 $\pm$ 0.2 & 25 $\pm$ 0 & 7.7 $\pm$ 0.1 \\
5&  11.88744 & -25.28881 & 219.4 $\pm$ 0.2 & 41 $\pm$ 0 & 8.6 $\pm$ 0.1 \\
6&  11.88755 & -25.28874 & 223.2 $\pm$ 0.5 & 30 $\pm$ 1 & 3.1 $\pm$ 0.1 \\
7SW&  11.88747 & -25.28868 & 241.6 $\pm$ 1.1 & 48 $\pm$ 2 & 1.8 $\pm$ 0.1 \\
7&  11.88757 & -25.28863 & 247.7 $\pm$ 0.5 & 37 $\pm$ 1 & 2.9 $\pm$ 0.1 \\
& & & 30